In [23]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'





In [24]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [25]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [26]:
#usage
powers = [42,44,46]
num_antennas_BS   = 4
pop_size = 20
print(f"{BOLD}POWERS IN DBM{RESET}")
population = generate_random_solutions(powers, num_antennas_BS, pop_size)
print(population)


POWERS IN DBM
[[42 42 46 44]
 [46 42 42 44]
 [46 42 42 44]
 [42 44 44 44]
 [44 44 44 46]
 [42 46 42 44]
 [44 44 44 46]
 [46 44 46 42]
 [44 44 46 44]
 [44 46 42 42]
 [46 44 46 46]
 [42 44 46 46]
 [44 46 46 44]
 [46 44 44 46]
 [46 44 46 46]
 [42 44 46 42]
 [46 42 44 44]
 [44 46 46 46]
 [42 46 46 42]
 [44 46 42 46]]


In [27]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [28]:
#usage
print(f"{BOLD}POWERS IN LINEAR{RESET}")
linear_pop=calculate_linear_trans_power(population)
print(linear_pop)

POWERS IN LINEAR
[[15.84893192 15.84893192 39.81071706 25.11886432]
 [39.81071706 15.84893192 15.84893192 25.11886432]
 [39.81071706 15.84893192 15.84893192 25.11886432]
 [15.84893192 25.11886432 25.11886432 25.11886432]
 [25.11886432 25.11886432 25.11886432 39.81071706]
 [15.84893192 39.81071706 15.84893192 25.11886432]
 [25.11886432 25.11886432 25.11886432 39.81071706]
 [39.81071706 25.11886432 39.81071706 15.84893192]
 [25.11886432 25.11886432 39.81071706 25.11886432]
 [25.11886432 39.81071706 15.84893192 15.84893192]
 [39.81071706 25.11886432 39.81071706 39.81071706]
 [15.84893192 25.11886432 39.81071706 39.81071706]
 [25.11886432 39.81071706 39.81071706 25.11886432]
 [39.81071706 25.11886432 25.11886432 39.81071706]
 [39.81071706 25.11886432 39.81071706 39.81071706]
 [15.84893192 25.11886432 39.81071706 15.84893192]
 [39.81071706 15.84893192 25.11886432 25.11886432]
 [25.11886432 39.81071706 39.81071706 39.81071706]
 [15.84893192 39.81071706 39.81071706 15.84893192]
 [25.11886432 

In [29]:
# fitness function  ==== linear_power_vect
def sum_linear_powers(array):
    return np.sum(array, axis=1)

In [30]:
# usage
print(f"{BOLD}SUM OF LINEAR POWERS{RESET}")
Lin_vector = sum_linear_powers(linear_pop)
print(*Lin_vector, sep='\n')

SUM OF LINEAR POWERS
96.62744521966779
96.62744521966779
96.62744521966779
91.20552486989861
115.16731000063717
96.62744521966779
115.16731000063717
120.58923035040634
115.16731000063716
96.62744521966779
144.5510154811449
120.58923035040635
129.85916274089104
129.85916274089104
144.5510154811449
96.62744521966779
105.89737761015247
144.5510154811449
111.31929795992166
120.58923035040635


In [31]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

Linear noise: [3.98107171e-15]


In [32]:
#calculate the bit rate matrix
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [33]:
#Usage 
print(f"{BOLD}BITRATES(formula){RESET}")
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

BITRATES(formula)
[[16679522.44733575 15839412.18480891 18148113.71781224 16929792.79569887]
 [18008285.4123791  15839412.18480891 16819349.98857859 16929792.79569887]
 [18008285.4123791  15839412.18480891 16819349.98857859 16929792.79569887]
 [16679522.44733575 16503788.7231917  17483731.0036872  16929792.79569887]
 [17343902.99389214 16503788.7231917  17483731.0036872  17594174.15009605]
 [16679522.44733575 17168168.61267186 16819349.98857859 16929792.79569887]
 [17343902.99389214 16503788.7231917  17483731.0036872  17594174.15009605]
 [18008285.4123791  16503788.7231917  18148113.71781224 16265413.93559985]
 [17343902.99389214 16503788.7231917  18148113.71781224 16929792.79569887]
 [17343902.99389214 17168168.61267186 16819349.98857859 16265413.93559985]
 [18008285.4123791  16503788.7231917  18148113.71781224 17594174.15009605]
 [16679522.44733575 16503788.7231917  18148113.71781224 17594174.15009605]
 [17343902.99389214 17168168.61267186 18148113.71781224 16929792.79569887]
 [18008

In [34]:
# USAGE
print(f"{BOLD}SUM OF BITRATES{RESET}")
row_sum_bitrate = summing(bitrate)
print(*row_sum_bitrate, sep='\n')


SUM OF BITRATES


NameError: name 'summing' is not defined

In [35]:
def selection_half_population(size_pop, pop_size_dbm, row_sums):
    L = size_pop // 2
    
    # Combine the population and their corresponding row sums
    help_fun = np.column_stack((pop_size_dbm, row_sums))

    # Sort the combined array in ascending order of row sums
    sorted_help_fun = help_fun[np.argsort(help_fun[:, -1])]
    
    # Select the top half of the population
    parent = sorted_help_fun[:L, :-1]
    parent_EE = sorted_help_fun[:L, -1]
    
    # Combine the selected parents and their corresponding row sums
    selected_parents = [[list(parent_row), parent_EE_value] for parent_row, parent_EE_value in zip(parent, parent_EE)]
    
    return selected_parents


In [37]:
# Use the selection_half_population function
selected_parents = selection_half_population(pop_size, population, Lin_vector)

# Print the output
print("Selected Parents:")
print(*selected_parents, sep='\n')


Selected Parents:
[[42.0, 44.0, 44.0, 44.0], 91.20552486989861]
[[42.0, 42.0, 46.0, 44.0], 96.62744521966779]
[[42.0, 44.0, 46.0, 42.0], 96.62744521966779]
[[42.0, 46.0, 42.0, 44.0], 96.62744521966779]
[[44.0, 46.0, 42.0, 42.0], 96.62744521966779]
[[46.0, 42.0, 42.0, 44.0], 96.62744521966779]
[[46.0, 42.0, 42.0, 44.0], 96.62744521966779]
[[46.0, 42.0, 44.0, 44.0], 105.89737761015247]
[[42.0, 46.0, 46.0, 42.0], 111.31929795992166]
[[44.0, 44.0, 46.0, 44.0], 115.16731000063716]


In [38]:
def extract_powers_in_dbm(selected_parents):
    powers = [list(parent[0]) for parent in selected_parents]
    return powers

In [40]:
only_dbm=extract_powers_in_dbm(selected_parents)
print(*only_dbm, sep='\n')

[42.0, 44.0, 44.0, 44.0]
[42.0, 42.0, 46.0, 44.0]
[42.0, 44.0, 46.0, 42.0]
[42.0, 46.0, 42.0, 44.0]
[44.0, 46.0, 42.0, 42.0]
[46.0, 42.0, 42.0, 44.0]
[46.0, 42.0, 42.0, 44.0]
[46.0, 42.0, 44.0, 44.0]
[42.0, 46.0, 46.0, 42.0]
[44.0, 44.0, 46.0, 44.0]


In [1]:
# def corresponding_row_sum_bitrate_v2(selected_parents, pop_size_dbm, row_sum_bitrate):
#     parent_indices = [pop_size_dbm.tolist().index(parent[0]) for parent in selected_parents]
#     corresponding_row_sum_bitrate_values = [row_sum_bitrate[index] for index in parent_indices]
    
#     return corresponding_row_sum_bitrate_values

# # Usage
# corresponding_bitrate_values_v2 = corresponding_row_sum_bitrate_v2(selected_parents, population, row_sum_bitrate)
# print("Corresponding  Bitrate (Exact Order):")
# print(*corresponding_bitrate_values_v2, sep='\n')


In [42]:
def one_point_crossover_first_second(parents):
    offspring = []
    
    # Perform crossover between the first two parents
    parent1 = parents[0][0]
    parent2 = parents[1][0]
    crossover_point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    offspring.append(offspring1)
    offspring.append(offspring2)

    # Perform crossover between the second two parents
    parent3 = parents[2][0]
    parent4 = parents[3][0]
    crossover_point = random.randint(1, len(parent3) - 1)
    offspring3 = parent3[:crossover_point] + parent4[crossover_point:]
    offspring4 = parent4[:crossover_point] + parent3[crossover_point:]
    offspring.append(offspring3)
    offspring.append(offspring4)
    
    
    parent5 = parents[4][0]
    parent6 = parents[5][0]
    crossover_point = random.randint(1, len(parent5) - 1)
    offspring5 = parent5[:crossover_point] + parent6[crossover_point:]
    offspring6 = parent6[:crossover_point] + parent5[crossover_point:]
    offspring.append(offspring5)
    offspring.append(offspring6)
    
    parent7 = parents[6][0]
    parent8 = parents[7][0]
    crossover_point = random.randint(1, len(parent7) - 1)
    offspring7 = parent7[:crossover_point] + parent8[crossover_point:]
    offspring8 = parent8[:crossover_point] + parent7[crossover_point:]
    offspring.append(offspring7)
    offspring.append(offspring8)
    
    parent9 = parents[8][0]
    parent10 = parents[9][0]
    crossover_point = random.randint(1, len(parent9) - 1)
    offspring9 = parent9[:crossover_point] + parent10[crossover_point:]
    offspring10 = parent10[:crossover_point] + parent9[crossover_point:]
    offspring.append(offspring9)
    offspring.append(offspring10)
    
    

    return offspring

# Usage
crossover_offspring = one_point_crossover_first_second(selected_parents)
print("Crossover Offspring:")
print(*crossover_offspring, sep='\n')
print(type(crossover_offspring))


Crossover Offspring:
[42.0, 42.0, 46.0, 44.0]
[42.0, 44.0, 44.0, 44.0]
[42.0, 46.0, 42.0, 44.0]
[42.0, 44.0, 46.0, 42.0]
[44.0, 46.0, 42.0, 44.0]
[46.0, 42.0, 42.0, 42.0]
[46.0, 42.0, 42.0, 44.0]
[46.0, 42.0, 44.0, 44.0]
[42.0, 46.0, 46.0, 44.0]
[44.0, 44.0, 46.0, 42.0]
<class 'list'>


In [43]:
def mutation(offspring, powers):
    mutated_offspring = []
    for child in offspring:
        mutated_child = child.copy()
        mutation_position = random.randint(0, len(child) - 1)
        new_gene = random.choice(powers)
        
        while new_gene == mutated_child[mutation_position]:
            new_gene = random.choice(powers)
        
        mutated_child[mutation_position] = new_gene
        mutated_offspring.append(mutated_child)
    return mutated_offspring

# Usage
mutated_offspring = mutation(crossover_offspring, powers)
print("Mutated Offspring:")
print(*mutated_offspring, sep='\n')


Mutated Offspring:
[42.0, 42.0, 46.0, 42]
[42.0, 46, 44.0, 44.0]
[46, 46.0, 42.0, 44.0]
[44, 44.0, 46.0, 42.0]
[44.0, 44, 42.0, 44.0]
[46.0, 42.0, 44, 42.0]
[46.0, 42.0, 42.0, 46]
[46.0, 44, 44.0, 44.0]
[42.0, 46.0, 42, 44.0]
[44.0, 44.0, 42, 42.0]


In [44]:
def concatenate_parents_and_offspring(selected_parents, mutated_offspring):
    combined_population = []

    # Add selected parents to the combined population
    for parent, parent_EE_value in selected_parents:
        combined_population.append(parent)

    # Add mutated offspring to the combined population
    for offspring in mutated_offspring:
        combined_population.append(np.array(offspring))

    # Convert the combined population list to a NumPy array
    combined_population = np.array(combined_population)

    return combined_population

# Usage
combined_population = concatenate_parents_and_offspring(selected_parents, mutated_offspring)
print("Combined Population:")
print(combined_population)


Combined Population:
[[42. 44. 44. 44.]
 [42. 42. 46. 44.]
 [42. 44. 46. 42.]
 [42. 46. 42. 44.]
 [44. 46. 42. 42.]
 [46. 42. 42. 44.]
 [46. 42. 42. 44.]
 [46. 42. 44. 44.]
 [42. 46. 46. 42.]
 [44. 44. 46. 44.]
 [42. 42. 46. 42.]
 [42. 46. 44. 44.]
 [46. 46. 42. 44.]
 [44. 44. 46. 42.]
 [44. 44. 42. 44.]
 [46. 42. 44. 42.]
 [46. 42. 42. 46.]
 [46. 44. 44. 44.]
 [42. 46. 42. 44.]
 [44. 44. 42. 42.]]
